In [ ]:
#@title Check current GPU assigned
!nvidia-smi -L
!nvidia-smi

In [ ]:
#@title 1. install / import packages (mandatory)
try:
  #install
  !sudo apt install python3-venv wget
  !pip install pyngrok

  #import
  from IPython.utils import capture
  from IPython.display import clear_output
  from subprocess import getoutput
  import ipywidgets as widgets
  import os
  import sys
  import fileinput
  import time
  import base64
  import re
  import gdown
  from gdown.download import get_url_from_gdrive_confirmation
  import requests
  from urllib.request import urlopen, Request
  from urllib.parse import urlparse, parse_qs, unquote
  from tqdm import tqdm
  import six
  from google.colab import drive
  from pyngrok import ngrok, conf

  #needed to transform name of repo
  salt=base64.b64decode(("ZWJ1aQ==").encode('ascii')).decode('ascii')

  #functions
  def inf(msg, style, wdth):
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)

  clear_output()
  inf('\u2714 Done','success', '50px')
except Exception as e:
  print("Error while installing / importing packages: ", str(e))
  inf('\u2718 Failed','danger', '30px')

In [ ]:
#@title 2. Initiate a1111 directory (mandatory)
try:
  use_temp_storage = True #@param {type:"boolean"}
  #@markdown - If not, make sure you have enough space on your gdrive as drive will be mounted.

  a1111_ai_folder = "sd" #@param {type:"string"}
  #@markdown - name of your existing folder or new folder to be created.

  if not use_temp_storage:
    drive_path = '/content/drive'
    drive.mount(drive_path)
    a1111_root_path = drive_path + '/MyDrive/' + a1111_ai_folder
  else:
    a1111_root_path = '/content/' + a1111_ai_folder

  if not os.path.exists(a1111_root_path):
    print("path " + a1111_root_path + " does not exist. creating directory.")
    %mkdir $a1111_root_path
  else:
    print("existing directory found.")

  %cd $a1111_root_path

  clear_output()
  inf('\u2714 Done','success', '50px')
except Exception as e:
  print("Error while setting up directory", str(e))
  inf('\u2718 Failed','danger', '30px')

In [ ]:
#@title 3. initiate required functions (mandatory)
try:
  def get_src(url):
      parsed_url = urlparse(url)
      if parsed_url.netloc == 'civitai.com':
          src='civitai'
      elif parsed_url.netloc == 'drive.google.com':
          src='gdrive'
      elif parsed_url.netloc == 'huggingface.co':
          src='huggingface'
      else:
          src='others'
      return src

  def get_name(url, gdrive):
      if not gdrive:
          response = requests.get(url, allow_redirects=False)
          if "Location" in response.headers:
              redirected_url = response.headers["Location"]
              quer = parse_qs(urlparse(redirected_url).query)
              if "response-content-disposition" in quer:
                  disp_val = quer["response-content-disposition"][0].split(";")
                  for vals in disp_val:
                      if vals.strip().startswith("filename="):
                          filenm=unquote(vals.split("=", 1)[1].strip())
                          return filenm.replace("\"","")
      else:
          headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
          lnk="https://drive.google.com/uc?id={id}&export=download".format(id=url[url.find("/d/")+3:url.find("/view")])
          res = requests.session().get(lnk, headers=headers, stream=True, verify=True)
          res = requests.session().get(get_url_from_gdrive_confirmation(res.text), headers=headers, stream=True, verify=True)
          content_disposition = six.moves.urllib_parse.unquote(res.headers["Content-Disposition"])
          filenm = re.search(r"filename\*=UTF-8''(.*)", content_disposition).groups()[0].replace(os.path.sep, "_")
          return filenm

  def download(url, dst, msg):
      file_size = None
      req = Request(url, headers={"User-Agent": "torch.hub"})
      u = urlopen(req)
      meta = u.info()
      if hasattr(meta, 'getheaders'):
          content_length = meta.getheaders("Content-Length")
      else:
          content_length = meta.get_all("Content-Length")
      if content_length is not None and len(content_length) > 0:
          file_size = int(content_length[0])

      with tqdm(total=file_size, disable=False, mininterval=0.5,
              bar_format=msg+' |{bar:20}| {percentage:3.0f}%') as pbar:
          with open(dst, "wb") as f:
              while True:
                  buffer = u.read(8192)
                  if len(buffer) == 0:
                      break
                  f.write(buffer)
                  pbar.update(len(buffer))
              f.close()


  def get_sd_model(ver):

      if ver=='SD 1.5':
          model_path= a1111_root_path+'/stable-diffusion-w'+salt+'/models/Stable-diffusion/v1-5-pruned-emaonly.safetensors'
          model_link='https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors'
      elif ver=='SDXL':
          model_path= a1111_root_path+'/stable-diffusion-w'+salt+'/models/Stable-diffusion/sd_xl_base_1.0.safetensors'
          model_link='https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors'

      if not os.path.exists(model_path):
          !gdown --fuzzy -O $model_path $model_link
          if os.path.exists(model_path):
              clear_output()
              inf('\u2714 Done','success', '50px')
          else:
              inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
          clear_output()
          inf('\u2714 Model already exists','primary', '300px')

      return model_path

  clear_output()
  inf('\u2714 Done','success', '50px')
except Exception as e:
  print("Error while initiating functions: ", str(e))
  inf('\u2718 Failed','danger', '30px')

In [ ]:
#@title 4. Install or Update a1111 (mandatory)

try:
  initiate = 'Install' # @param ["Install", "Update", "Bypass"] {type:"raw"}
  #@markdown - **Install** - if you are running for first time, irrespective of gdrive or temp storage
  #@markdown - **Update** - if you want to update already installed code. (e.g. if you used gdrive instead of temp storage)
  #@markdown - **Bypass** - if you want to bypass the install or update and skip to next part of processing.

  if not os.path.exists(a1111_root_path):
    raise Exception("root directory not found.")

  if initiate == "Install":
    with capture.capture_output() as cap:
      !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-w$salt

  elif initiate == "Update":
    with capture.capture_output() as cap:
      %cd stable-diffusion-w$salt
      !git reset --hard
      !git checkout master
      time.sleep(1)
      !rm webui.sh
      !git pull
      %cd ..

  inf('\u2714 Done','success', '50px')
except Exception as e:
  print("Error while fetching code", str(e))
  inf('\u2718 Failed','danger', '30px')

In [ ]:
#@title 5. Model Download / Load if exists **(Mandatory for 1st time)**

try:
    #@markdown Select from available models for initial download
    model_version = "SD 1.5" #@param ["SDXL", "SD 1.5"]

    #@markdown Or

    #@markdown Insert the full path of your custom model or to a folder containing multiple models
    path_to_model = "" #@param {type:"string"}

    #@markdown Or

    #@markdown Enter a link to model
    model_link = "" #@param {type:"string"}

    # driving script
    if path_to_model !='':
        if os.path.exists(str(path_to_model)):
            inf('\u2714 Using the custom model.','success', '200px')
        else:
            inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
            raise Exception("path does not exist.")

        model_path=path_to_model

    elif model_link != "":

        src=get_src(model_link)

        if src=='civitai':
            modelname=get_name(model_link, False)
            model_path= a1111_root_path + f'/stable-diffusion-w{salt}/models/Stable-diffusion/{modelname}'
            if not os.path.exists(model_path):
                download(model_link, model_path, 'Downloading the custom model')
                clear_output()
            else:
                inf('\u2714 Model already exists','primary', '300px')
        elif src=='gdrive':
            modelname=get_name(model_link, True)
            model_path= a1111_root_path + f'/stable-diffusion-w{salt}/models/Stable-diffusion/{modelname}'
            if not os.path.exists(model_path):
                gdown.download(url=model_link, output=model_path, quiet=False, fuzzy=True)
                clear_output()
            else:
                inf('\u2714 Model already exists','primary', '300px')
        else:
            modelname=os.path.basename(model_link)
            model_path= a1111_root_path + f'/stable-diffusion-w{salt}/models/Stable-diffusion/{modelname}'
            if not os.path.exists(model_path):
                gdown.download(url=model_link, output=model_path, quiet=False, fuzzy=True)
                clear_output()
            else:
                inf('\u2714 Model already exists','primary', '700px')

        if os.path.exists(model_path):
            inf('\u2714 Model downloaded, using the given link.','success', '300px')
        else:
            !rm model_path
            inf('\u2718 Wrong link, check that the link is valid','danger', "300px")

    else:
        model_path=get_sd_model(model_version)

    clear_output()
    inf('\u2714 Done','success', '50px')
except:
  print("Error occurred while downloading")
  inf('\u2718 Failed','danger', '30px')

#@markdown ---

In [ ]:
#@title 6. Download LoRA
try:
  lora_link = "" #@param {type:"string"}

  lora_folder = a1111_root_path +'/stable-diffusion-w'+salt+'/models/Lora'
  os.makedirs(lora_folder, exist_ok=True)

  src=get_src(lora_link)

  if src=='civitai':
      modelname=get_name(lora_link, False)
      lora_model_path= lora_folder + f'/{modelname}'
      if not os.path.exists(lora_model_path):
        download(lora_link, lora_model_path, 'Downloading the LoRA model '+modelname)
        clear_output()
      else:
        inf('\u2714 Model already exists','primary', '300px')
  elif src=='gdrive':
      modelname=get_name(lora_link, True)
      lora_model_path= lora_folder +f'/{modelname}'
      if not os.path.exists(lora_model_path):
        gdown.download(url=lora_link, output=lora_model_path, quiet=False, fuzzy=True)
        clear_output()
      else:
        inf('\u2714 Model already exists','primary', '300px')
  else:
      modelname=os.path.basename(lora_link)
      lora_model_path=lora_folder +f'/{modelname}'
      if not os.path.exists(lora_model_path):
        gdown.download(url=lora_link, output=lora_model_path, quiet=False, fuzzy=True)
        clear_output()
      else:
        inf('\u2714 Model already exists','primary', '700px')

  if os.path.exists(lora_model_path) :
    inf('\u2714 LoRA downloaded','success', '300px')
  else:
    inf('\u2718 Wrong link, check that the link is valid','danger', "300px")
except:
  print("Error occurred while downloading")
  inf('\u2718 Failed','danger', '30px')

#@markdown ---

In [ ]:
#@title 7. Download ControlNet

try:
  from torch.hub import download_url_to_file
  from subprocess import run

  cn_model_link = "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth" #@param {type:"string"}
  #@markdown - Download/update ControlNet extension and its models
  cn_models_dir=a1111_root_path+'/stable-diffusion-w'+salt+'/extensions/sd-w'+salt+'-controlnet/models'

  def download_cn_model(url, model_dir):
      filename = os.path.basename(urlparse(url).path)
      pth = os.path.join(model_dir, filename)
      if not os.path.exists(pth):
          print('Downloading: '+os.path.basename(url))
          download_url_to_file(url, pth, hash_prefix=None, progress=True)
      else:
        print(f"[1;32mThe model {filename} already exists[0m")

  with capture.capture_output() as cap:
    !git clone https://github.com/Mikubill/sd-w$salt-controlnet $a1111_root_path/stable-diffusion-w$salt/extensions/sd-w$salt-controlnet

  with capture.capture_output() as cap:
    cfgname=os.path.basename(urlparse(cn_model_link).path).split('.')[0]+'.yaml'
    # %cd $cn_models_dir
    run(['cp', cn_models_dir + '/' + 'cldm_v21.yaml', cn_models_dir + '/' + cfgname])
    # %cd $a1111_root_path

  if cn_model_link != "":
    download_cn_model(cn_model_link, cn_models_dir)

  clear_output()
  inf('\u2714 Done','success', '50px')

except Exception as e:
  print("Error occurred while downloading", str(e))
  inf('\u2718 Failed','danger', '30px')

  #@markdown ---

In [ ]:
#@title 8. start tunnel
try:
  ngrok_token = "" #@param {type:"string"}
  #@markdown - Input your ngrok token if you want to use ngrok server

  ngrok_port = 7860 #@param {type:"number"}
  #@markdown - Input port for ngrok server

  if ngrok_token!="":
      ngrok.kill()
      url=ngrok.connect(ngrok_port, pyngrok_config=conf.PyngrokConfig(auth_token=ngrok_token) , bind_tls=True).public_url
      print("Interface will start at the following given link soon after you launch, refresh the page once launch is successful:\n",url)

  inf('\u2714 Done','success', '50px')
except Exception as e:
  print("Error while starting tunnel", str(e))
  inf('\u2718 Failed','danger', '30px')

In [ ]:
#@title 9. launch a1111 (mandatory)

try:

  ckptdir=''
  if os.path.exists(a1111_root_path + f'/stable-diffusion-w{salt}/models/Stable-diffusion'):
    ckptdir='--ckpt-dir ' + a1111_root_path + f'/stable-diffusion-w{salt}/models/Stable-diffusion'

  !cd stable-diffusion-w$salt
  !python launch.py --api --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --upcast-sampling $ckptdir
except Exception as e:
  print("Error while launching a1111", str(e))
  inf('\u2718 Failed','danger', '30px')